In [1]:
import os
import langchain_chroma
import openai
from openai import OpenAI
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import psycopg2
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
POST_DB_HOST = os.getenv("POST_DB_HOST")
POST_DB_NAME = os.getenv("POST_DB_NAME")
POST_DB_USER = os.getenv("POST_DB_USER")
POST_DB_PASSWD = os.getenv("POST_DB_PASSWD")
POST_DB_PORT = os.getenv("POST_DB_PORT")
# PostgreSQL 연결 엔진 생성
post_engine = create_engine(f'postgresql://{POST_DB_USER}:{POST_DB_PASSWD}@{POST_DB_HOST}:{POST_DB_PORT}/{POST_DB_NAME}')
db = psycopg2.connect(host=POST_DB_HOST, dbname=POST_DB_NAME,user=POST_DB_USER,password=POST_DB_PASSWD,port=POST_DB_PORT)
cursor = db.cursor()
db.autocommit = False

import os
# 데이터 불러오기
# 이력서 불러오기
path = "./data/빅데이터AI_이력서.pdf"
doc = fitz.open(path)
for page in doc:
    resume_text = page.get_text()
    print(resume_text)

📄 이력서 (Resume)
[기본 정보]
이름: 홍길동
연락처: 010-1234-5678
이메일: honggildong.ai@gmail.com
주소: 서울특별시 강남구 테헤란로 123
[학력]
고려대학교 컴퓨터학과 졸업 (2018.03 ~ 2024.02)
GPA: 3.85 / 4.5
관련 과목: 머신러닝, 데이터마이닝, 통계학, 빅데이터처리, 딥러닝 이론과 실습
[기술 스택]
Programming: Python, SQL, R
Frameworks/Libraries: Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy
Tools: Jupyter, Git, Docker, Tableau
DBMS: MySQL, MongoDB, Hadoop(HDFS), Spark
Cloud: Google Colab, AWS EC2 & S3 (기초 수준)
[프로젝트 경험]
1. 신문 기사 기반 감성 분석 모델 개발 (2023.03 ~ 2023.06)
자연어처리(NLP) 기반 감성 분류 모델 개발
KoNLPy와 Scikit-learn을 이용한 전처리 및 모델 학습
정확도 86% 달성
2. 머신러닝 기반 개인 맞춤형 영화 추천 시스템 (2023.09 ~ 2023.12)
📄 이력서 (Resume)
1

Content-based Filtering 및 Collaborative Filtering 기법 적용
Streamlit으로 웹 인터페이스 구현
kaggle 데이터셋 기반, Precision@10: 0.73
[자격증]
ADsP (데이터분석 준전문가) – 2023.08
SQLD (SQL 개발자) – 2024.01
📄 이력서 (Resume)
2



In [5]:
df = pd.read_csv("../data_backup/rec_data.csv")

In [2]:
from elasticsearch import Elasticsearch
from langchain.vectorstores import ElasticVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.storage import InMemoryStore

In [4]:
es = Elasticsearch("http://localhost:9200")

In [7]:
df

,Unnamed: 0,rec_idx,chunk,company_nm,recruit_kewdcdnm,company_place,career,education
0,0,46254462,[3500/중식제공]식품 쇼핑몰 MD 경력직 채용\n\n채용공고 상세,(주)다온미트,['온라인MD'],경기 하남시,경력(년수무관) · 정규직,학력무관
1,1,46552267,iOS 개발자 경력 모집\n\n채용공고 상세\nIOS 개발자 모집모집부문 및 상세내...,한국전자인증㈜,['Objective-C'],서울 서초구,1 ~ 10년 · 정규직,학력무관
2,2,46809948,인하우스 디자인 / 시각 제품 그래픽 산업 3D UX UI 웹\n\n채용공고 상세\...,(주)아트앤매니지먼트,"['크리에이터', '광고디자인', '그래픽디자인', '디지털디자인', '산업디자인']",서울 강남구 외,신입 · 경력 · 정규직,학력무관
3,3,46858808,제주 드림타워 복합리조트 [카지노&호텔] 대규모 신입 공개 채용 채용공고 상세\n ...,(주)엘티엔터테인먼트,"['출납', '사무직', '문서작성', '비품관리', '사무보조']",제주 제주시 외,신입 · 정규직 외,학력무관
4,4,47037243,경영지원팀-경영지원(신입-경력)사원 구인 공고\n\n채용공고 상세\n빠르게 성장하는...,엑사옵토닉스(주),"['경리', '총무', '경영지원', '회계']",경기 화성시,경력무관 · 정규직,학력무관
...,...,...,...,...,...,...,...,...
13848,13848,50283723,[동인광학] 부설연구소 전자 FW/HW 개발자 모집\n\n채용공고 상세\n[동인광학...,(주)동인광학,"['연구원', 'R&D']",경기 부천시 오정구,경력 3년↑ · 정규직,"대학(2,3년)↑"
13849,13849,50283724,생산직 직원 모집합니다\n\n채용공고 상세\n안녕하세요 \n당사는 내화채움재등을 제...,주식회사더원에프에스,"['생산관리', 'MES', '생산']",경기 양주시,경력무관 · 정규직,학력무관
13850,13850,50283728,광주 에이치원 호텔 프론트 데스크/업무 직원\n\n채용공고 상세\n광주 에이치원 호...,(주)에이치원호텔,"['호텔리어', '객실관리', '고객안내', '고객응대', '프론트']",광주 북구,경력무관 · 정규직,고졸↑
13851,13851,50283729,경북어울림)포항시발달장애인주간활동센터 제공인력 채용\n\n채용공고 상세\n경북어울림...,경북어울림사회적협동조합,"['사회복지사', '생활복지사', '장애인복지']",경북 포항시 북구,경력무관 · 정규직 외,학력무관


In [8]:
from langchain.schema import Document

documents = []
for _, row in df.iterrows():
    doc = Document(
        page_content=row["chunk"],
        metadata={
            "rec_idx": row["rec_idx"],
            "company_nm": row["company_nm"],
            "recruit_kewdcdnm": row["recruit_kewdcdnm"],
            "company_place": row["company_place"],
            "career": row["career"],
            "education": row["education"],
        }
    )
    documents.append(doc)


In [16]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema.document import Document
import os

# 1. OpenAI 임베딩 모델
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. Chroma에서 벡터 리트리버 로딩 (이미 구축된 벡터 DB)
vectordb = Chroma(
    persist_directory="./chroma_data",
    embedding_function=embedding_model,
    collection_name="chroma_test"
)
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# 3. BM25Retriever (chunk 리스트 그대로 사용)
# 👉 'chunk' 컬럼과 metadata로 Document 객체 생성
from langchain.schema import Document

bm25_docs = [
    Document(page_content=row["chunk"], metadata=row.drop("chunk").to_dict())
    for _, row in df.iterrows()
]
bm25_retriever = BM25Retriever.from_documents(bm25_docs)
bm25_retriever.k = 5

# 4. 하이브리드 리트리버 조합
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.8, 0.2]  # → 가중치 조정 가능 (BM25 강화하려면 [0.7, 0.3]처럼)
)


In [34]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 정의
template = """
당신은 인재매칭 AI 어시스턴트입니다. 사용자 이력서에 기반하여 채용공고를 5개 추천해주세요.

- 반드시 이력서에 기반할 것.
- 채용공고 상세는 반드시 모두 출력할 것.
- 출력 시 채용공고의 양식을 사용할 것.
- 채용공고 추천 이유를 한줄로 설명할 것.                            
- 채용공고 전체 내용을 기반하여 분석할 것. 제목만 보고 판단하지 말 것.
- page_content의 전체 텍스트를 기준으로 판단할 것.

### 출력 형식 예시
1. **직무명**: AI 분석가 (신입)
- 공고 번호                           
- **산업 분야**: 
- **필요 기술**: 
- **채용공고 상세**:
- **추천 사유**:

이 정보를 바탕으로 사용자 정보에 가장 적합한 채용 공고 5개를 추천해주세요.
출력 형식과 동일한 형식으로 출력해주세요.                                          

채용공고:
{context}

이력서:
{question}
"""

prompt = PromptTemplate.from_template(template)

# LLM 세팅
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

# 전체 체인 정의
rag_chain = (
     {
        "context": hybrid_retriever | (lambda docs: "\n\n".join(
            f"{doc.metadata.get('rec_idx')}\n{doc.metadata.get('page')} \n{doc.page_content.strip()}"
            for doc in docs
        )),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [31]:
doc

Document(metadata={'rec_idx': 50283731, 'company_nm': '아피아엔지니어링', 'recruit_kewdcdnm': "['설계엔지니어', '기계설계', '부품설계', '시스템설계', '2D설계']", 'company_place': '경남 진주시 외', 'career': '경력무관 · 정규직 외', 'education': '대학(2,3년)↑'}, page_content='CATIA 부품설계 설계자 채용\n\n채용공고 상세\nCATIA 부품설계 설계자 채용2007년 04월 05일에 설립된 기타 엔지니어링 서비스업업종의 항공기부품설계사업을 하는 중소기업,주식회사기업 입니다.모집부문 및 상세내용공통 자격요건ㆍ학력 : 전문대졸 이상ㆍ전공 : 기계공학계열CATIA 설계설계팀 0명담당업무ㆍ주요업무직무내용ㆍ항공기 또는 발전설비 부품설계자격요건ㆍAutoCAD 사용가능자(필수)ㆍCATIA 사용 능력 우수자 우대우대사항ㆍ관련 직무 근무경험 보유자 우대근무조건ㆍ근무형태:정규직(수습기간)-3개월, 계약직(정규직 전환가능)ㆍ근무일시:주 5일(월~금) 08:00~17:00*근무예정지 : 경남 사천시 or 경남 창원시 성산구ㆍ근무지역:(52818) 경남 진주시 동부로169번길 12 윙스타워 A동 310호(충무공동)전형절차 서류전형 실무자 면접 임원면접 최종합격접수기간 및 방법ㆍ접수기간:2025년 3월 19일 (수) 14시~ 2025년 3월 30일 (일) 24시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.')

In [35]:
# 사용 예시
question = resume_text
answer = rag_chain.invoke(question)
print(answer)

1. **직무명**: 보조기기 서비스 담당자 (계약직)
   - 공고 번호: 50195947
   - **산업 분야**: 비영리재단, 사회복지
   - **필요 기술**: 보조공학, 작업치료, 사회복지, 3D프린트 설계
   - **채용공고 상세**: 서울시서북보조기기센터에서 보조기기 서비스(상담, 평가, 적용 및 현장서비스), 소모품 구매 및 관리, 사업 홍보 및 홍보컨텐츠 제작 등의 업무를 담당할 계약직 인력을 모집합니다. 관련 학과 졸업자 및 졸업 예정자, 보조기기센터 및 비영리법인, 사회복지시설 관련 업무 경험자 우대.
   - **추천 사유**: 사회복지 및 보조기기 관련 경험이 있는 지원자에게 적합한 직무입니다.

2. **직무명**: 품질연구원 (품질경영본부)
   - 공고 번호: 50224035
   - **산업 분야**: 환경, 건설, 기술
   - **필요 기술**: 신제품 개발, 기술 특허 출원 및 관리, 연구예산 및 기술전략 관리
   - **채용공고 상세**: 뉴보텍에서 신제품개발(R&D), 기술 특허 출원 및 관리, 연구예산 및 기술전략 관리 등의 업무를 담당할 품질연구원을 모집합니다. 화공학 전공 및 관련 업계 종사자, 관련 직무 유 경험자 우대.
   - **추천 사유**: 연구개발 및 기술 관리에 관심이 있는 지원자에게 적합한 직무입니다.

3. **직무명**: SW개발자 (정규직)
   - 공고 번호: 50275838
   - **산업 분야**: 소프트웨어 개발
   - **필요 기술**: React.js, Java, Spring framework, Node.js, Python, RESTful API
   - **채용공고 상세**: 중소기업에서 프론트엔드 및 백엔드 웹 애플리케이션 개발, 데이터베이스 설계 및 최적화, RESTful API 설계 및 구현 등의 업무를 담당할 SW개발자를 모집합니다. 경력 4년 이상, 관련 기술 경험자 우대.
   - **추천 사유**: 다양한 개발 기술 스택을 보유한 지원자에게 적합한 직무입니다.